#### Importando librerias.

In [1]:
from wostools import CollectionLazy
import igraph
import pandas as pd
import statistics as st

#### Importando datos (consulta).

In [3]:
collection = CollectionLazy.from_filenames("records.txt")
citation_pairs = collection.citation_pairs()
citation_pairs = list(citation_pairs)

### Organizando grafo.

In [4]:
vertices = set()
for a, b in citation_pairs:
    vertices.add(a)
    vertices.add(b)

vertices = list(vertices)

In [5]:
graph = igraph.Graph(directed=True)
graph.add_vertices(vertices)
graph.add_edges(citation_pairs)

In [6]:
graph = graph.components(mode="weak").giant()
graph = graph.simplify()

In [7]:
selected = graph.vs.select(
    lambda node: not (node.indegree() == 1 and node.outdegree() == 0)
)
graph = graph.subgraph(selected)

In [8]:
graph_undirected = graph.as_undirected()
max_blondel = max(
    graph_undirected.community_multilevel(weights=None, return_levels=False).sizes()
)
for cluster in graph_undirected.community_multilevel(
    weights=None, return_levels=False
):
    if len(cluster) == max_blondel:
        cluster_blondel = graph_undirected.subgraph(cluster)

#### Atributos.

In [9]:
features = pd.DataFrame(index=["records.txt"])

for i in graph,cluster_blondel:
    if i == graph:
        j = 'Complet'
    else:
        j = 'Blondel_C'
    nodes = i.vcount()
    edges = i.ecount()
    features['Nodes '+ j] = nodes
    features['Edges ' + j] = edges

    max_degree = i.maxdegree()
    degree = i.degree()
    betweenness = i.betweenness()
    closeness = i.closeness()

    diameter = i.diameter()
    density = i.density()
    clusters_biconnected = len(i.biconnected_components())
    features['Diameter '+ j] = diameter
    features['Density '+ j] = density
    features['Cluster biconected '+ j] = clusters_biconnected

    mean_degree = st.mean(degree)
    sd_degree = st.stdev(degree)
    var_degree = st.variance(degree)
    mean_indegree = st.mean(i.indegree())
    mean_outdegree = st.mean(i.outdegree())
    features['Max degree '+ j] = max_degree
    features['Sd degree '+ j] = sd_degree
    features['Var degree '+ j] = var_degree
    features['Mean indegree '+ j] = mean_indegree
    features['Mean outdegree '+ j] = mean_outdegree

    mean_betweenness = st.mean(betweenness)
    sd_betw = st.stdev(betweenness)
    var_betw = st.variance(betweenness)
    features['Mean betweenness '+ j] = mean_betweenness
    features['Sd betweenness '+ j] = sd_betw
    features['var betweenness '+ j] = var_betw

    mean_closeness = st.mean(closeness)
    sd_closeness = st.stdev(closeness)
    var_closeness = st.variance(closeness)
    features['Mean closeness '+ j] = mean_closeness
    features['Sd closeness '+ j] = sd_closeness
    features['Var closeness '+ j] = var_closeness

In [10]:
features

,Nodes Complet,Edges Complet,Diameter Complet,Density Complet,Cluster biconected Complet,Max degree Complet,Sd degree Complet,Var degree Complet,Mean indegree Complet,Mean outdegree Complet,...,Sd degree Blondel_C,Var degree Blondel_C,Mean indegree Blondel_C,Mean outdegree Blondel_C,Mean betweenness Blondel_C,Sd betweenness Blondel_C,var betweenness Blondel_C,Mean closeness Blondel_C,Sd closeness Blondel_C,Var closeness Blondel_C
records.txt,2742,10197,6,0.001357,13,279,11.860383,140.668674,3.718818,3.718818,...,4.776501,22.814961,3.57037,3.57037,662.2,1450.327091,2.103449e+06,0.238376,0.033113,0.001096
